# AER - Access Entitlement Review Reporter (Optimized)

**Features:**
- Interactive Login (Browser)
- Robust Batch Processing (Skips Audit Logs, Non-stop on errors)
- Detailed Status Reporting (Approved/Denied counts)
- Split Export (Individual App Files + Consolidated File)

In [3]:
# === Cell 1: 設定與認證 (互動式視窗登入 & UTF-8 Logging) ===
import os
import sys
import io
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication
import requests

# 讀取 .env
load_dotenv()

# === Logging 設定 (支援 Windows + Jupyter/IPython) ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear() # 避免重複執行時累積 handler
logger.setLevel(logging.INFO)

def _get_console_stream():
    """
    取得可用的 console stream:
    - 一般 Python (非 Jupyter) 優先使用 .reconfigure 或包裝 buffer 為 UTF-8。
    - 在 Jupyter/IPython (OutStream 無 .buffer/.reconfigure)，回傳 None，
      讓 logging.StreamHandler 使用預設 (通常是 sys.stderr)，避免拋錯。
    """
    s = getattr(sys, "stdout", None)
    # 嘗試使用 reconfigure (多數非 IPython 情境可行)
    try:
        if s and hasattr(s, "reconfigure"):
            s.reconfigure(encoding="utf-8")
            # 同時調整 stderr 以保持一致
            if hasattr(sys.stderr, "reconfigure"):
                sys.stderr.reconfigure(encoding="utf-8")
            return s
    except Exception:
        pass

    # 若有底層 buffer, 包裝成 UTF-8 TextIOWrapper
    try:
        buf = getattr(s, "buffer", None)
        if buf:
            return io.TextIOWrapper(buf, encoding="utf-8", errors="replace")
    except Exception:
        pass
        
    # Jupyter/IPython: 回退為 None, 交由 logging 使用預設 stream
    return None

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

# Console handler
console_stream = _get_console_stream()
ch = logging.StreamHandler(console_stream) # 在 notebook 中不要強制替換 stream
ch.setFormatter(formatter)
logger.addHandler(ch)

# File handler: 明確使用 UTF-8
fh = logging.FileHandler(log_file, encoding="utf-8")
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD 設定 ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
# CLIENT_SECRET 不需要 (Public client / Delegated)

# === SharePoint 設定 ===
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review/Q4/AVIDXCHANGE-TEST"
BASE_PATH = APP_NAME

# === Email 設定 ===
SENDER_EMAIL = os.getenv("SENDER_EMAIL")

# === Delegated Scopes ===
SCOPES = ["Sites.Read.All", "User.Read.All", "Mail.Send"]

# === 初始化 MSAL Public Client ===
app = PublicClientApplication(
    CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}"
)

def _set_headers_from_result(result: dict) -> dict:
    """
    依據 MSAL 取得的 result 設定 Authorization header。
    回傳 headers dict，若失敗則拋出例外以便上層處理。
    """
    if not result or "access_token" not in result:
        err = result.get("error")
        desc = result.get("error_description")
        raise RuntimeError(f"登入失敗: {err} - {desc}")
    token = result["access_token"]
    return {"Authorization": f"Bearer {token}"}

# === 互動式瀏覽器登入 ===
# 提醒: App Registration 需設定 redirect URI: http://localhost
print("🚀 正在開啟瀏覽器進行登入...")
interactive_result = app.acquire_token_interactive(
    scopes=SCOPES,
    prompt="select_account",      # 允許使用者切換身分
)

headers = _set_headers_from_result(interactive_result)
logger.info("✅ 互動式登入成功")
logger.info(f"🎉 使用者: {interactive_result.get('id_token_claims', {}).get('name')}")
logger.info(f"處理 app: {APP_NAME}")
logger.info(f"Log 檔案: {log_file}")

🚀 正在開啟瀏覽器進行登入...


2026-01-08 10:08:10,540 | INFO | ✅ 互動式登入成功
2026-01-08 10:08:10,542 | INFO | 🎉 使用者: david shih
2026-01-08 10:08:10,542 | INFO | 處理 app: 2025 Entitlement Review/Q4/AVIDXCHANGE-TEST
2026-01-08 10:08:10,543 | INFO | Log 檔案: output/aer_20260108_100807.log


In [4]:
# === Cell 2: SharePoint 函數庫 (API) ===

def get_site_id(site_name: str) -> str:
    """取得 SharePoint site ID"""
    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_HOST}:/sites/{site_name}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["id"]


def list_folders(site_id: str, path: str) -> list[dict]:
    """列出資料夾下的所有子資料夾（自動判斷是否為根目錄）"""
    if not path or path.strip() == "":
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root/children"
    else:
        clean_path = path.strip("/")
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    return [
        {"name": item["name"], "webUrl": item.get("webUrl", "")}
        for item in resp.json().get("value", [])
        if item.get("folder")
    ]


def list_excel_files(site_id: str, folder_path: str) -> list[dict]:
    """列出資料夾下的 Excel 檔案"""
    clean_path = folder_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    files = []
    for item in resp.json().get("value", []):
        if item["name"].endswith(".xlsx"):
            files.append({
                "id": item["id"],
                "name": item["name"],
                "lastModifiedDateTime": item.get("lastModifiedDateTime"),
                "webUrl": item.get("webUrl")
            })
    return sorted(files, key=lambda f: f.get("lastModifiedDateTime", ""), reverse=True)


def download_file(site_id: str, file_path: str) -> bytes:
    """下載檔案內容"""
    clean_path = file_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/content"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.content


def get_file_audit_log(site_id: str, file_id: str) -> str:
    """取得檔案版本歷史 (Audit Log)"""
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{file_id}/versions"
    resp = requests.get(url, headers=headers)
    
    if resp.status_code != 200:
        return "無法取得版本紀錄"

    versions = resp.json().get("value", [])
    logs = []
    for v in versions:
        mod_time = v.get("lastModifiedDateTime", "")[:19].replace("T", " ")
        user_info = v.get("lastModifiedBy", {}).get("user", {})
        app_info = v.get("lastModifiedBy", {}).get("application", {})
        actor = user_info.get("displayName") or app_info.get("displayName") or "Unknown System"
        ver_num = v.get("id", "")
        logs.append(f"{mod_time} - {actor} (v{ver_num})")
        
    return "\n".join(logs)

# 重新測試連線
try:
    site_id = get_site_id(SITE_NAME)
    logger.info(f"SharePoint 連線成功，Site ID: {site_id[:20]}...")
except Exception as e:
    logger.error(f"SharePoint 連線失敗: {e}")
    raise

2026-01-08 10:08:19,813 | INFO | SharePoint 連線成功，Site ID: davidshih.sharepoint...


In [ ]:
# === Cell 2.5: 資料夾選擇器 (UI) ===
import ipywidgets as widgets
from IPython.display import display, clear_output

current_path = "" 
selected_target_path = None

def create_folder_browser():
    if 'list_folders' not in globals() or 'site_id' not in globals():
        print("⚠️ 請先執行 Cell 2 以載入 SharePoint 函式！")
        return

    # UI Elements
    path_label = widgets.HTML(value=f"<b>目前路徑:</b> Root (Shared Documents)")
    folder_dropdown = widgets.Dropdown(options=[], description='子資料夾:', disabled=False, layout=widgets.Layout(width='50%'))
    btn_enter = widgets.Button(description='📂 進入', button_style='info', icon='folder-open')
    btn_back = widgets.Button(description='⬅️ 返回', button_style='', icon='arrow-left')
    btn_select = widgets.Button(description='✅ 設定為目標資料夾', button_style='success', icon='check')
    msg_output = widgets.Output()

    def load_folders(path):
        try:
            items = list_folders(site_id, path)
            options = [item['name'] for item in items if item['name'] != "Forms"]
            folder_dropdown.options = sorted(options)
            
            if not options:
                folder_dropdown.disabled = True
                folder_dropdown.placeholder = "無子資料夾"
            else:
                folder_dropdown.disabled = False
                folder_dropdown.value = options[0] if options else None
        except Exception as e:
            with msg_output:
                print(f"❌ 讀取失敗: {e}")

    def on_enter_click(b):
        global current_path
        if folder_dropdown.value:
            if current_path:
                current_path = f"{current_path}/{folder_dropdown.value}"
            else:
                current_path = folder_dropdown.value
            path_label.value = f"<b>目前路徑:</b> {current_path}"
            load_folders(current_path)

    def on_back_click(b):
        global current_path
        if "/" in current_path:
            current_path = current_path.rsplit("/", 1)[0]
        else:
            current_path = ""
        path_label.value = f"<b>目前路徑:</b> {current_path or 'Root (Shared Documents)'}"
        load_folders(current_path)

    def on_select_click(b):
        global BASE_PATH
        BASE_PATH = current_path
        with msg_output:
            clear_output()
            print(f"🎯 已鎖定目標路徑: {BASE_PATH}")
            print("👉 請繼續執行 Cell 3 和 Cell 4。")

    btn_enter.on_click(on_enter_click)
    btn_back.on_click(on_back_click)
    btn_select.on_click(on_select_click)

    load_folders(current_path)

    ui = widgets.VBox([
        widgets.HTML("<h3>📂 SharePoint 資料夾瀏覽器</h3>"),
        path_label,
        widgets.HBox([btn_back, folder_dropdown, btn_enter]),
        widgets.HTML("<br>"),
        btn_select,
        msg_output
    ])
    display(ui)

create_folder_browser()

In [11]:
# === Cell 3: Excel 讀取（處理 AutoFilter visible rows）===
from openpyxl import load_workbook
from io import BytesIO

COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"

def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    """
    讀取 Excel 的 visible rows（考慮 AutoFilter）
    """
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"找不到必要欄位。現有欄位: {list(col_map.keys())}")
    
    has_filter = any(
        ws.row_dimensions[i].hidden 
        for i in range(2, ws.max_row + 1) 
        if i in ws.row_dimensions
    )
    
    results = []
    for row_idx in range(2, ws.max_row + 1):
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        
        row = [cell.value for cell in ws[row_idx]]
        
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        if not has_filter:
            if reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower():
                continue
        
        is_missing = response is None or str(response).strip() == ""
        
        results.append({
            "reviewer": reviewer_name,
            "response": response,
            "details": details,
            "is_missing": is_missing,
            "row_number": row_idx,
            "file_name": file_name,
            "folder_url": folder_url
        })
    
    wb.close()
    return results

logger.info("Excel 讀取函數已載入")

2026-01-08 10:11:07,398 | INFO | Excel 讀取函數已載入


In [12]:
# === Cell 4: 資料收集 (邏輯修正版：含 Category 偵測 & 容錯收集) ===
import pandas as pd
import os

logger.info(f"目前設定路徑: {BASE_PATH}")

# 1. 取得第一層資料夾
try:
    initial_folders = list_folders(site_id, BASE_PATH)
except Exception as e:
    logger.error(f"無法讀取初始路徑: {e}")
    initial_folders = []

# 過濾掉 "Audit Log" 或系統資料夾
valid_folders = [
    f for f in initial_folders 
    if "audit log" not in f["name"].lower() and f["name"] not in ["Forms", "_private"]
]

target_app_folders = [] # 格式: (Category, App名稱, 完整路徑)

if not valid_folders:
    logger.warning("❌ 目標路徑下沒有有效的資料夾！")
else:
    # 判斷是否為 Batch Mode (Q4/Q2)
    first_folder_name = valid_folders[0]["name"]
    first_folder_path = f"{BASE_PATH}/{first_folder_name}"
    
    # 嘗試抓取目前路徑的最後一部分作為 "Category" (例如 Q2, Q4)
    current_category_name = BASE_PATH.strip("/").split("/")[-1] if BASE_PATH else "Root"

    sub_items = list_folders(site_id, first_folder_path)
    is_batch_mode = len(sub_items) > 0 
    
    if is_batch_mode:
        logger.info(f"🔄 偵測到 Batch Mode (Category: {current_category_name})")
        for f in valid_folders:
            app_sub_path = f"{BASE_PATH}/{f['name']}"
            # 在 Batch Mode 下，Category 就是當前資料夾名稱 (如 Q4)，App 就是子資料夾名
            target_app_folders.append((current_category_name, f['name'], app_sub_path))
    else:
        logger.info(f"✅ 偵測到 Single App Mode")
        # Single Mode 下，Category 設為 "Single" 或抓上層
        app_name = BASE_PATH.split("/")[-1] if "/" in BASE_PATH else APP_NAME
        parent_cat = BASE_PATH.split("/")[-2] if "/" in BASE_PATH and len(BASE_PATH.split("/")) > 1 else "Manual"
        target_app_folders.append((parent_cat, app_name, BASE_PATH))

# === 開始跑迴圈 ===
all_responses = []
errors = []

for category, current_app_name, current_path in target_app_folders:
    logger.info(f"🚀 開始處理 App: {current_app_name} (Category: {category})")
    
    # --- Try/Except 包在 App 層級，確保單一 App 失敗不影響整體 ---
    try:
        # 抓取該 App 下的所有審核人
        reviewers = list_folders(site_id, current_path)
        
        if not reviewers:
             logger.warning(f"  [警示] App 資料夾是空的: {current_app_name}")
             continue

        for folder in reviewers:
            reviewer_name = folder["name"]
            
            # 跳過 Audit Log
            if "audit log" in reviewer_name.lower(): 
                continue
            if reviewer_name in ["Forms", "_private"]:
                continue
                
            folder_url = folder["webUrl"]
            folder_path = f"{current_path}/{reviewer_name}"
            
            try:
                # 列出 Excel 檔案
                excel_files = list_excel_files(site_id, folder_path)
                
                # 尋找目標檔案
                target_files = [f for f in excel_files if reviewer_name.lower() in f["name"].lower()]
                
                if not target_files:
                    valid_excels = [f for f in excel_files if not f["name"].startswith("~$")]
                    if valid_excels:
                        target_file = valid_excels[0] 
                    else:
                        continue
                else:
                    target_file = target_files[0]
                
                file_name = target_file["name"]
                file_id = target_file["id"]
                file_download_path = f"{folder_path}/{file_name}"
                
                # 抓取 Audit Log
                audit_log_str = get_file_audit_log(site_id, file_id)
                
                # 下載並解析
                excel_bytes = download_file(site_id, file_download_path)
                rows = read_visible_rows(excel_bytes, reviewer_name, file_name, folder_url)
                
                # 補上 App Name 與 Category 欄位
                for row in rows:
                    row["Category"] = category
                    row["App_Name"] = current_app_name
                    row["Audit_History"] = audit_log_str
                    row["Last_Modified"] = target_file.get("lastModifiedDateTime")
                
                all_responses.extend(rows)
                # logger.info(f"  Added {len(rows)} from {reviewer_name}")

            except Exception as e:
                # 捕捉單一審核人的錯誤 (不影響同 App 其他人)
                logger.error(f"  ❌ 審核人處理失敗 {reviewer_name}: {e}")
                errors.append({
                    "Category": category,
                    "App_Name": current_app_name, 
                    "reviewer": reviewer_name, 
                    "error": str(e), 
                    "folder_url": folder_url
                })

    except Exception as e:
        # 捕捉整個 App 的錯誤 (不影響其他 App)
        logger.error(f"❌ 嚴重錯誤 - 跳過整個 App ({current_app_name}): {e}")
        errors.append({
            "Category": category,
            "App_Name": current_app_name, 
            "reviewer": "Entire App Failed", 
            "error": str(e), 
            "folder_url": "N/A"
        })

# 轉成 DataFrame
df = pd.DataFrame(all_responses)
logger.info(f"\n=== 收集完成 ===")
logger.info(f"總計: {len(all_responses)} 筆記錄")
if len(df) > 0:
    logger.info(f"缺漏: {df['is_missing'].sum()} 筆")

2026-01-08 10:11:11,552 | INFO | 目前設定路徑: 2025 Entitlement Review/Q4
2026-01-08 10:11:12,255 | INFO | 🔄 偵測到 Batch Mode (Category: Q4)
2026-01-08 10:11:12,256 | INFO | 🚀 開始處理 App: AVIDXCHANGE-TEST (Category: Q4)
2026-01-08 10:11:16,401 | INFO | 🚀 開始處理 App: Visa (Category: Q4)
2026-01-08 10:11:20,575 | INFO | 
=== 收集完成 ===
2026-01-08 10:11:20,576 | INFO | 總計: 26 筆記錄
2026-01-08 10:11:20,577 | INFO | 缺漏: 7 筆


In [13]:
# === Cell 5: 互動式狀態表格 (新增詳細狀態統計) ===
from IPython.display import display, HTML
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

def format_summary_excel(file_path: str):
    """美化摘要表"""
    try:
        wb = load_workbook(file_path)
        ws = wb.active
        for column_cells in ws.columns:
            header_cell = column_cells[0]
            col_letter = get_column_letter(header_cell.column)
            max_length = 0
            for cell in column_cells:
                try:
                    if cell.value:
                        cell_len = len(str(cell.value))
                        if "http" in str(cell.value) and cell_len > 50: cell_len = 50
                        if cell_len > max_length: max_length = cell_len
                except: pass
            adjusted_width = (max_length + 2) * 1.1
            if adjusted_width > 50: adjusted_width = 50
            ws.column_dimensions[col_letter].width = adjusted_width
        wb.save(file_path)
    except Exception as e:
        logger.warning(f"美化失敗: {e}")

def create_status_table(df: pd.DataFrame) -> pd.DataFrame:
    if len(df) == 0:
        display(HTML("<p style='color: red;'>沒有資料可顯示</p>"))
        return pd.DataFrame()
    
    # 確保 response 欄位是字串，避免錯誤
    df['response_clean'] = df['response'].fillna("").astype(str).str.lower().str.strip()

    # --- 統計邏輯 ---
    # 1. 基礎統計 (Total, Missing)
    base_summary = df.groupby(["Category", "App_Name", "reviewer", "folder_url"]).agg(
        total=("reviewer", "count"),
        missing=("is_missing", "sum")
    ).reset_index()

    # 2. 詳細狀態統計 (Approved, Denied, etc.)
    # 使用 Pivot Table 計算各狀態數量
    status_counts = pd.pivot_table(
        df, 
        index=["Category", "App_Name", "reviewer"], 
        columns="response_clean", 
        aggfunc="size", 
        fill_value=0
    ).reset_index()

    # 對應常見關鍵字 (根據 Excel 實際內容調整)
    def get_count(row, keywords):
        count = 0
        for col in status_counts.columns:
            if col in ["Category", "App_Name", "reviewer"]: continue
            # 檢查 column header 是否包含關鍵字
            if any(k in col for k in keywords):
                count += row[col]
        return count

    # 合併回去
    summary = pd.merge(base_summary, status_counts, on=["Category", "App_Name", "reviewer"], how="left")
    
    # 為了避免欄位不存在報錯，手動計算並新增欄位
    summary['Count_Approved'] = summary.apply(lambda r: get_count(r, ['approv', 'retain', 'keep']), axis=1)
    summary['Count_Denied'] = summary.apply(lambda r: get_count(r, ['denied', 'remove', 'delete', 'revoke']), axis=1)
    summary['Count_Change'] = summary.apply(lambda r: get_count(r, ['change', 'modif']), axis=1)

    summary["completion"] = ((summary["total"] - summary["missing"]) / summary["total"] * 100).round(1)
    
    # 為了兼容 Email 寄信 Cell，組合 Display_App_Name
    summary["Display_App_Name"] = summary["Category"] + " / " + summary["App_Name"]

    # 排序
    summary_sorted = summary.sort_values(["Category", "missing", "App_Name"], ascending=[True, False, True])
    
    # --- 匯出 Excel ---
    summary_file = f"output/summary_status_all_{timestamp}.xlsx"
    
    # 選取要輸出的欄位
    display_cols = [
        "Category", "App_Name", "reviewer", "completion", "missing", "total",
        "Count_Approved", "Count_Denied", "Count_Change", "folder_url"
    ]
    
    summary_sorted[display_cols].to_excel(summary_file, index=False)
    format_summary_excel(summary_file)
    
    # --- 顯示 HTML ---
    def make_row(r):
        bg_color = "#ffcccc" if r["missing"] > 0 else "#ccffcc"
        return f"""
        <tr style="background-color: {bg_color};">
            <td>{r['Category']}</td>
            <td style="font-weight:bold;">{r['App_Name']}</td>
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['completion']}%</td>
            <td><b style='color:red'>{int(r['missing'])}</b> / {int(r['total'])}</td>
            <td style='font-size:0.9em; color:#555;'>
                ✅{int(r['Count_Approved'])} | ❌{int(r['Count_Denied'])} | ⚠️{int(r['Count_Change'])}
            </td>
        </tr>
        """
    
    html = f"""
    <style>
        .status-table {{ border-collapse: collapse; width: 100%; font-family: sans-serif; font-size: 14px; }}
        .status-table th, .status-table td {{ border: 1px solid #ddd; padding: 6px; text-align: left; }}
        .status-table th {{ background-color: #4CAF50; color: white; }}
    </style>
    <h3>📊 審核狀態總覽 (含詳細分類)</h3>
    <p>📥 摘要 Excel: <code>{summary_file}</code></p>
    <table class="status-table">
        <tr>
            <th>Category</th>
            <th>App Name</th>
            <th>Reviewer</th>
            <th>Completion</th>
            <th>Missing / Total</th>
            <th>Details (Appr | Deny | Chg)</th>
        </tr>
        {''.join(summary_sorted.apply(make_row, axis=1))}
    </table>
    """
    display(HTML(html))
    return summary_sorted

summary_df = create_status_table(df)

Category,App Name,Reviewer,Completion,Missing / Total,Details (Appr | Deny | Chg)
Q4,AVIDXCHANGE-TEST,Joe Petti,50.0%,2 / 4,✅2 | ❌0 | ⚠️0
Q4,Visa,Charlie Brown,60.0%,2 / 5,✅2 | ❌1 | ⚠️0
Q4,Visa,Joe Petti,50.0%,2 / 4,✅2 | ❌0 | ⚠️0
Q4,AVIDXCHANGE-TEST,Jane Smith,80.0%,1 / 5,✅3 | ❌0 | ⚠️0
Q4,AVIDXCHANGE-TEST,Bob O'Brien,100.0%,0 / 3,✅3 | ❌0 | ⚠️0
Q4,Visa,Alice Wonderland,100.0%,0 / 5,✅1 | ❌4 | ⚠️0


In [14]:
# === Cell 6: 發送提醒郵件 (Interactive Mail) ===
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from urllib.parse import quote
import time

def get_user_email(name: str) -> str:
    try:
        clean_name = name.split("(")[0].strip()
        encoded_name = quote(clean_name)
        url = f"https://graph.microsoft.com/v1.0/users?$filter=displayName eq '{encoded_name}'&$select=userPrincipalName,mail"
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200 and resp.json().get("value"):
            user = resp.json()["value"][0]
            return user.get("mail") or user.get("userPrincipalName")
        url = f"https://graph.microsoft.com/v1.0/users?$filter=startswith(displayName, '{encoded_name}')&$select=userPrincipalName,mail"
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200 and resp.json().get("value"):
            user = resp.json()["value"][0]
            return user.get("mail") or user.get("userPrincipalName")
    except Exception:
        pass
    return ""

def send_mail_api(sender: str, to: str, subject: str, body: str) -> tuple[bool, str]:
    try:
        url = f"https://graph.microsoft.com/v1.0/users/{sender}/sendMail"
        data = {
            "message": {
                "subject": subject,
                "body": {"contentType": "HTML", "content": body},
                "toRecipients": [{"emailAddress": {"address": to}}]
            },
            "saveToSentItems": "true"
        }
        resp = requests.post(url, headers={**headers, "Content-Type": "application/json"}, json=data)
        if resp.status_code == 202:
            return True, "已發送"
        else:
            return False, f"失敗 ({resp.status_code}): {resp.text}"
    except Exception as e:
        return False, str(e)

if 'summary_df' not in dir() or len(summary_df) == 0:
    display(widgets.HTML("<h3 style='color:red'>⚠️ 請先執行前面的 Cell 產生 summary_df</h3>"))
else:
    targets = summary_df[summary_df['missing'] > 0].copy()
    if len(targets) == 0:
        display(widgets.HTML("<h3 style='color:green'>🎉 恭喜！所有審核都已完成，無需寄信。</h3>"))
    else:
        default_sender = SENDER_EMAIL if 'SENDER_EMAIL' in globals() and SENDER_EMAIL else "me"
        style = {'description_width': 'initial'}
        
        global_subject = widgets.Text(value="[Action Required] 2025 Access Review Pending", description="<b>郵件主旨:</b>", layout=widgets.Layout(width='95%'), style=style)
        default_template = """<p>Hi {name},</p><p>溫馨提醒：您在 <b>{app}</b> 還有 <b>{missing}</b> 筆權限審核尚未完成。</p><p>請點擊連結進行作業：<a href="{link}">開啟資料夾</a></p><p>謝謝</p>"""
        global_body_template = widgets.Textarea(value=default_template, description="<b>內容模板:</b>", placeholder="使用 {name}, {app}, {missing}, {link} 作為變數", layout=widgets.Layout(width='95%', height='100px'), style=style)
        btn_apply_template = widgets.Button(description="套用模板到下方所有列", button_style='info', icon='arrow-down')
        
        header_html = widgets.HTML(
            """<div style="display: flex; font-weight: bold; background-color: #f0f0f0; padding: 8px; border-bottom: 2px solid #ccc;"><div style="width: 30px;">選</div><div style="width: 150px;">App / Reviewer</div><div style="width: 200px;">收件人 (To)</div><div style="width: 200px;">寄件人 (From)</div><div style="flex: 1;">郵件內容 (可編輯)</div><div style="width: 80px;">操作</div></div>"""
        )

        row_widgets_list = []
        row_data_map = {}
        
        print("⏳ 正在從 Azure AD 搜尋 Email，請稍候...")
        
        for idx, row in targets.iterrows():
            r_name = row['reviewer']
            r_app = row['Display_App_Name'] # Cell 5 有補上這個欄位
            r_miss = int(row['missing'])
            r_link = row['folder_url']
            
            found_email = get_user_email(r_name)
            initial_body = default_template.format(name=r_name, app=r_app, missing=r_miss, link=r_link)
            
            w_check = widgets.Checkbox(value=True, indent=False, layout=widgets.Layout(width='30px'))
            w_info = widgets.HTML(f"<div style='font-size:12px; line-height:1.2'><b>{r_app}</b><br>{r_name}<br><span style='color:red'>缺 {r_miss} 筆</span></div>", layout=widgets.Layout(width='150px'))
            w_to = widgets.Text(value=found_email, placeholder="輸入 Email", layout=widgets.Layout(width='190px'))
            w_from = widgets.Text(value=default_sender, placeholder="me 或 user@com...", layout=widgets.Layout(width='190px'))
            w_body = widgets.Textarea(value=initial_body, layout=widgets.Layout(width='98%', height='60px'))
            w_btn_send = widgets.Button(description="🚀", layout=widgets.Layout(width='40px'), button_style='warning', tooltip='單獨發送此封')
            w_status = widgets.Label(value="", layout=widgets.Layout(width='80px'))
            
            row_box = widgets.HBox([w_check, w_info, w_to, w_from, w_body, widgets.VBox([w_btn_send, w_status])], layout=widgets.Layout(border='1px solid #eee', padding='5px', align_items='center'))
            
            row_key = f"{r_app}_{r_name}"
            row_data_map[row_key] = {"check": w_check, "to": w_to, "from": w_from, "body": w_body, "status": w_status, "btn": w_btn_send, "data": {"name": r_name, "app": r_app, "missing": r_miss, "link": r_link}}
            row_widgets_list.append(row_box)
            
            def create_send_handler(key):
                def on_click(b):
                    widgets_row = row_data_map[key]
                    to_addr = widgets_row['to'].value.strip()
                    from_addr = widgets_row['from'].value.strip()
                    body_txt = widgets_row['body'].value
                    subject_txt = global_subject.value
                    if not to_addr:
                        widgets_row['status'].value = "❌ 無 Email"
                        return
                    widgets_row['btn'].disabled = True
                    widgets_row['status'].value = "⏳ 發送中..."
                    success, msg = send_mail_api(from_addr, to_addr, subject_txt, body_txt)
                    if success:
                        widgets_row['status'].value = "✅ 已寄出"
                        widgets_row['check'].value = False
                        widgets_row['btn'].button_style = 'success'
                    else:
                        widgets_row['status'].value = "❌ 失敗"
                        print(f"[{key}] 發送失敗: {msg}")
                        widgets_row['btn'].disabled = False
                return on_click
            w_btn_send.on_click(create_send_handler(row_key))

        clear_output()

        def on_apply_template(b):
            tpl = global_body_template.value
            for key, w_row in row_data_map.items():
                if w_row['check'].value:
                    d = w_row['data']
                    new_body = tpl.format(name=d['name'], app=d['app'], missing=d['missing'], link=d['link'])
                    w_row['body'].value = new_body
        btn_apply_template.on_click(on_apply_template)
        
        btn_send_all = widgets.Button(description="⚡ 發送已選取的郵件 (Bulk Send)", button_style='danger', layout=widgets.Layout(width='100%'), icon='paper-plane')
        log_output = widgets.Output()
        
        def on_send_all(b):
            with log_output:
                clear_output()
                count = 0
                for key, w_row in row_data_map.items():
                    if w_row['check'].value:
                        w_row['btn'].click()
                        count += 1
                        time.sleep(0.5)
                print(f"已執行 {count} 封郵件的發送程序。")

        btn_send_all.on_click(on_send_all)

        ui = widgets.VBox([
            widgets.HTML("<h3>📧 郵件發送控制台</h3>"),
            global_subject,
            widgets.HBox([global_body_template, widgets.VBox([widgets.Label(" "), btn_apply_template])]),
            widgets.HTML("<hr>"),
            header_html,
            widgets.VBox(row_widgets_list, layout=widgets.Layout(max_height='500px', overflow_y='auto')),
            widgets.HTML("<hr>"),
            btn_send_all,
            log_output
        ])
        display(ui)

In [9]:
# === Cell 7: 產生結案報告 (個別檔案 + 合併檔案) ===
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import pandas as pd
from IPython.display import display, HTML
import re

def format_excel_layout(file_path: str):
    """Excel 美化: 自動換行與調整欄寬"""
    try:
        wb = load_workbook(file_path)
        ws = wb.active
        # 定義長文字欄位 (包含你要求的 Reason for Change / Details)
        long_text_cols = [
            "Audit_History", "Details of Access change", "details", "Details", 
            "response", "error", "folder_url"
        ]
        
        for column_cells in ws.columns:
            header_cell = column_cells[0]
            header_val = str(header_cell.value).strip() if header_cell.value else ""
            col_letter = get_column_letter(header_cell.column)
            
            if header_val in long_text_cols:
                ws.column_dimensions[col_letter].width = 50
                for cell in column_cells:
                    cell.alignment = Alignment(wrap_text=True, vertical='top')
            else:
                max_length = 0
                for cell in column_cells:
                    try:
                        if cell.value:
                            val_len = len(str(cell.value))
                            if val_len > max_length: max_length = val_len
                    except: pass
                adj_width = min((max_length + 2) * 1.1, 60)
                ws.column_dimensions[col_letter].width = adj_width
                
        wb.save(file_path)
    except Exception as e:
        logger.warning(f"Format error: {e}")

def clean_filename(s):
    return re.sub(r'[\\/*?:"<>|]', "_", str(s))

print("--- 正在產生報告 ---")
report_html = "<h3>📂 檔案輸出清單</h3><ul>"

# 定義輸出的欄位 (加入 Details / Reason for Change)
export_cols = [
    "Category", "App_Name", "reviewer", 
    "response", "details", 
    "is_missing", 
    "Audit_History", "Last_Modified", 
    "file_name", "folder_url"
]

if len(df) > 0:
    # 1. 產生 [合併總表] (The Big File)
    consolidated_file = f"output/ALL_APPS_Consolidated_{timestamp}.xlsx"
    valid_cols = [c for c in export_cols if c in df.columns]
    
    df[valid_cols].to_excel(consolidated_file, index=False)
    format_excel_layout(consolidated_file)
    report_html += f"<li><b>🌎 合併總表 (Big File):</b> <code>{consolidated_file}</code></li>"
    
    # 2. 產生 [個別 App 檔案] (Individual Files)
    apps = df["App_Name"].unique()
    report_html += "<li>--- 個別 App 檔案 ---</li>"
    
    for app in apps:
        app_df = df[df["App_Name"] == app].copy()
        cat = app_df.iloc[0].get("Category", "Unknown")
        safe_cat = clean_filename(cat)
        safe_app = clean_filename(app)
        
        # 檔名格式: Category_AppName_Timestamp.xlsx
        app_filename = f"output/{safe_cat}_{safe_app}_{timestamp}.xlsx"
        app_df[valid_cols].to_excel(app_filename, index=False)
        format_excel_layout(app_filename)
        report_html += f"<li>{safe_cat} - {safe_app}: <code>{app_filename}</code></li>"

else:
    report_html += "<li style='color:red'>沒有成功讀取到任何資料。</li>"

# 3. 錯誤報告
if errors:
    error_df = pd.DataFrame(errors)
    error_file = f"output/ERRORS_{timestamp}.xlsx"
    error_df.to_excel(error_file, index=False)
    format_excel_layout(error_file)
    report_html += f"<li style='color:red'><b>⚠️ 錯誤報告:</b> <code>{error_file}</code></li>"

report_html += "</ul>"
display(HTML(report_html))

--- 正在產生報告 ---
